In [ ]:
import os
from glob import glob
import pandas as pd
import xlwings as xw
from tqdm import tqdm
from openpyxl import Workbook

In [ ]:
def low_count_filter(data, min_count=4, min_percentage=0.1):
    taxonomy_names = data.index
    chosen_list = []
    for taxo in taxonomy_names:
        taxo_data = data.loc[taxo].tolist()
        if len([x for x in taxo_data if x >= min_count]) / len(taxo_data) >= min_percentage:
            chosen_list.append(taxo)
    return data.loc[chosen_list].copy()

In [ ]:
def get_taxonomy(data, taxo="s"):
    all_taxo = data.index.tolist()
    selected_taxo = [x for x in all_taxo if "%s__" % taxo in x.split("|")[-1]]
    selected_data = data.loc[selected_taxo].copy()
    # selected_data.index = [x.split("%s__" % taxo)[-1] for x in selected_data.index]
    selected_data_sorted = selected_data.loc[selected_data.mean(1).sort_values(ascending=False).index.tolist()]
    return selected_data_sorted

In [ ]:
bracken_sammary_folder = R"D:\Project\gutDBase\bracken_summary_GSE"
mpa_combined_folder = R"D:\Project\gutDBase\mpa_combined"
metadata_folder = R"D:\Project\gutDBase\metadata"
host = 'human'

In [ ]:
mpa_PRJNA = [os.path.basename(mpa).split('_')[0]
             for mpa in glob(fR'{mpa_combined_folder}\{host}\*.txt')]

metadata_PRJNA = [os.path.basename(meta).split('.')[0] for meta in glob(
    fR'{metadata_folder}\{host}\*.csv')]

common_PRJNA = list(set(mpa_PRJNA) & set(metadata_PRJNA))

In [ ]:
def process(project: str):
    xlsxFile = os.path.join(bracken_sammary_folder, host,
                            f"{project}_summary.xlsx")
    metadataFile = os.path.join(metadata_folder, host, f"{project}.csv")
    if not os.path.exists(xlsxFile):
        wb = Workbook()
        wb.save(xlsxFile)

    xlapp = xw.App(add_book=False)  # 表示一个程序进程
    xlFile = xlapp.books.open(xlsxFile)
    mpaData = pd.read_csv(os.path.join(mpa_combined_folder, host,
                                       f"{project}_combined_mpa.txt"), sep="\t", index_col=0)
    mpaData_filtered = low_count_filter(mpaData)
    all_sheets = xlFile.sheets
    all_sheets_names = [x.name for x in all_sheets]
    if "metaData" not in all_sheets_names:
        sheet = xlFile.sheets["Sheet"]
        sheet.name = "metaData"
    if "raw" not in all_sheets_names:
        xlFile.sheets.add("raw", after="metaData")
    if "filtered" not in all_sheets_names:
        xlFile.sheets.add("filtered", after="raw")
    # write metadata
    metaData = pd.read_csv(metadataFile, index_col=0)
    sheet = xlFile.sheets["metaData"]
    sheet.clear_contents()
    sheet.range("A1").value = metaData
    sheet.autofit()
    sheet = xlFile.sheets["raw"]
    sheet.clear_contents()
    sheet.range("A1").value = mpaData
    sheet.autofit()
    sheet = xlFile.sheets["filtered"]
    sheet.clear_contents()
    sheet.range("A1").value = mpaData_filtered
    sheet.autofit()
    all_taxonomy = ["species", "genus", "family", "order", "class", "phylum"]
    for i in range(len(all_taxonomy)):
        taxonomy = all_taxonomy[i]
        after = "filtered" if i == 0 else all_taxonomy[i - 1]
        if taxonomy not in all_sheets_names:
            xlFile.sheets.add(taxonomy, after=after)
        mpaData_selected = get_taxonomy(mpaData_filtered, taxonomy[0])
        sheet = xlFile.sheets[taxonomy]
        sheet.clear_contents()
        sheet.range("A1").value = mpaData_selected
        sheet.autofit()
    sheet = xlFile.sheets["metaData"]
    sheet.activate()
    xlFile.save()
    xlapp.quit()

In [ ]:
for project in common_PRJNA:
    process(project)